In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../utils/")
sys.path.append("../")

In [3]:
import metrics
from agg_crossfit_results import *

In [4]:
pd.set_option('display.max_rows', 500)


In [5]:
set_name = 'third_set'
metrics_type = 'auc'

In [6]:
model_mapping = {
    'BART-Single': 1,
    'BART-Adapter-Single': 2,
    'BART-BiHNet-Single': 3,
    'BART-Adapter-Vanilla':4, 
    'BART-BiHNet+Vanilla':5,
    'BART-BiHNet+Reg':6,
    'BART-BiHNet+EWC':7,
    'BART-Adapter-Multitask':8,
    'BART-BiHNet-Multitask':9}

# Few shot

In [7]:
dir_dict = {"BART-BiHNet+Reg":f"../runs/BiHNet_Reg_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-BiHNet+EWC":f"../runs/BiHNet_ewc_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-Adapter-Multitask": f"../runs/adpter_mtl_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-BiHNet-Multitask": f"../runs/mtl_hnet_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-Adapter-Vanilla": f"../runs/adapter_vanilla_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-BiHNet+Vanilla": f"../runs/BiHNet_vanilla_{set_name}_0.01_s64_d256_limit/1e-4/0/",
#             "BART-Single": f"../runs/single_{set_name}_fs_0.01_s64_d256_limit/1e-4/0/",
#             "BART-Adapter-Single": f"../runs/adapter_single_{set_name}_fs_0.01_s64_d256_limit/1e-4/0/",
#             "BART-BiHNet-Single": f"../runs/BiHNet_single_{set_name}_fs_0.01_s64_d256_limit/1e-4/0/",
           }

In [8]:
def get_fs_df(dir_dict, metrics_type):
    dfs = []
    for alg in dir_dict:
        print(alg)
        for with_dash in [True, False]:
            df =  make_few_shot_result_table(dir_dict[alg], postfix="_naive_16shot", metrics_type=metrics_type, with_dash=with_dash)
            if df.empty:
                print("df", df)
                print(alg, with_dash, "hereeee")
                continue
            if "task_detail" in df.columns:
                df = pd.DataFrame(df.groupby(["task_name", "task_detail"])[metrics_type].max()).reset_index()
            else:
                df = pd.DataFrame(df.groupby(["task_name"])[metrics_type].max()).reset_index()
            df["alg"] = alg
            dfs.append(df)
    all_fs_df = pd.concat(dfs)
    all_fs_df = all_fs_df[[metrics_type, "task_name", "task_detail",  "alg"]]
    all_fs_df = all_fs_df.rename(columns={metrics_type: f'few-shot-{metrics_type}', 
                                                      'alg': 'model', 
                                                      'task_name': 'dataset', 
                                                      'task_detail': 'task'})
    return all_fs_df

In [9]:
fs_auc_df = get_fs_df(dir_dict, metrics_type='auc')
fs_f1_df = get_fs_df(dir_dict, metrics_type='f1')

BART-BiHNet+Reg
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-BiHNet+EWC
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-Adapter-Multitask
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-BiHNet-Multitask
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-Adapter-Vanilla
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-BiHNet+Vanilla
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-BiHNet+Reg
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-BiHNet+EWC
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-Adapter-Multitask
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-BiHNet-Multitask
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
BART-

In [10]:
fs_df = fs_f1_df.merge(fs_auc_df, on=['model', 'dataset', 'task'])

In [11]:
fs_df = fs_df.fillna("-")

In [13]:
fs_df.groupby("model")['few-shot-f1'].mean()

model
BART-Adapter-Multitask    0.329390
BART-Adapter-Vanilla      0.312395
BART-BiHNet+EWC           0.285581
BART-BiHNet+Reg           0.290309
BART-BiHNet+Vanilla       0.287627
BART-BiHNet-Multitask     0.272199
Name: few-shot-f1, dtype: float64

In [14]:
fs_df.groupby("model")['few-shot-auc'].mean()

model
BART-Adapter-Multitask    0.853153
BART-Adapter-Vanilla      0.828721
BART-BiHNet+EWC           0.819032
BART-BiHNet+Reg           0.808367
BART-BiHNet+Vanilla       0.814622
BART-BiHNet-Multitask     0.821467
Name: few-shot-auc, dtype: float64

In [15]:
fs_df['model_order'] = fs_df['model'].replace(model_mapping)


In [16]:
fs_df = fs_df.sort_values(['dataset','task', 'model_order'])

In [17]:
fs_df = fs_df[['dataset', 'task', 'model', 'few-shot-auc', 'few-shot-f1']]

## Latex Few shot

In [18]:
latex_table = fs_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{lllrr}
\toprule
           dataset &           task &                  model &  few-shot-auc &  few-shot-f1 \\
\midrule
              BAD2 &              - &   BART-Adapter-Vanilla &      0.664987 &     0.470588 \\
              BAD2 &              - &    BART-BiHNet+Vanilla &      0.582984 &     0.435165 \\
              BAD2 &              - &        BART-BiHNet+Reg &      0.660194 &     0.491484 \\
              BAD2 &              - &        BART-BiHNet+EWC &      0.633916 &     0.470588 \\
              BAD2 &              - & BART-Adapter-Multitask &      0.702097 &     0.514039 \\
              BAD2 &              - &  BART-BiHNet-Multitask &      0.714881 &     0.537445 \\
              BAD4 &              - &   BART-Adapter-Vanilla &      0.677866 &     0.454829 \\
              BAD4 &              - &    BART-BiHNet+Vanilla &      0.603978 &     0.454918 \\
              BAD4 &              - &        BART-BiHNet+Reg &      0.604742 &     0.447552 \\
         

# Final Accuracy

In [20]:
dir_dict = {"BART-BiHNet+Reg":f"../runs/BiHNet_Reg_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-BiHNet+EWC":f"../runs/BiHNet_ewc_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-Adapter-Multitask": f"../runs/adpter_mtl_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-BiHNet-Multitask": f"../runs/mtl_hnet_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-Adapter-Vanilla": f"../runs/adapter_vanilla_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-BiHNet+Vanilla": f"../runs/BiHNet_vanilla_{set_name}_0.01_s64_d256_limit/1e-4/0/",
           }

In [21]:
def get_final_df(dir_dict, metrics_type):
    dfs = []
    for alg in dir_dict:
        if alg =="pretrained" :
            continue
        print(alg)
        df1 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=True)
        df2 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=False)
        df = pd.concat([df1, df2])
        df['task_id'] = df['task_id'].astype(int)
        df['at_task'] = df['at_task'].astype(int)
        df['step'] = df['step'].astype(int)
        df = df[df.at_task == max(df.at_task)]
        if "task_detail" in  df.columns:
            idx = df.groupby(["task_name", "task_detail"])["step"].transform(max) == df["step"]
        else:
            print("here")
            idx = df.groupby(["task_name"])["step"].transform(max) == df["step"]

        df = df[idx]                      
        df["alg"] = alg
        dfs.append(df)
    all_final_df = pd.concat(dfs)
    all_final_df = all_final_df[[metrics_type, "task_name", "task_detail",  "alg"]]
    all_final_df = all_final_df.rename(columns={metrics_type: f'final-{metrics_type}', 
                                                      'alg': 'model', 
                                                      'task_name': 'dataset', 
                                                      'task_detail': 'task'})
    return all_final_df

In [22]:
final_auc_df = get_final_df(dir_dict, metrics_type='auc')
final_f1_df = get_final_df(dir_dict, metrics_type='f1')

BART-BiHNet+Reg
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-BiHNet+EWC
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-Adapter-Multitask
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-BiHNet-Multitask
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-Ad

In [23]:
final_df = final_auc_df.merge(final_f1_df, on=['model', 'dataset', 'task'])

In [24]:
final_df

,final-auc,dataset,task,model,final-f1
0,0.946478,jigsaw,obscene,BART-BiHNet+Reg,0.117117
1,0.682753,ucc,generalisation_unfair,BART-BiHNet+Reg,0.083832
2,0.918422,jigsaw,toxicity,BART-BiHNet+Reg,0.552076
3,0.841588,personal_attack,tpa,BART-BiHNet+Reg,0.072626
4,0.643099,cad,affiliationdirectedabuse,BART-BiHNet+Reg,0.129193
5,0.685203,ucc,generalisation,BART-BiHNet+Reg,0.096677
6,0.780278,ghc,hd,BART-BiHNet+Reg,0.308617
7,0.955545,hate,offensive,BART-BiHNet+Reg,0.781868
8,0.666253,abusive,hateful,BART-BiHNet+Reg,0.165092
9,0.681038,ucc,dismissive,BART-BiHNet+Reg,0.146835


In [25]:
final_df.groupby("model")['final-f1'].mean()

model
BART-Adapter-Multitask    0.397495
BART-Adapter-Vanilla      0.216248
BART-BiHNet+EWC           0.190391
BART-BiHNet+Reg           0.305835
BART-BiHNet+Vanilla       0.208114
BART-BiHNet-Multitask     0.318311
Name: final-f1, dtype: float64

In [26]:
final_df.groupby("model")['final-auc'].mean()

model
BART-Adapter-Multitask    0.875212
BART-Adapter-Vanilla      0.697841
BART-BiHNet+EWC           0.657128
BART-BiHNet+Reg           0.785940
BART-BiHNet+Vanilla       0.711536
BART-BiHNet-Multitask     0.832124
Name: final-auc, dtype: float64

## Instant Accuracy

In [34]:
def get_instant_df(dir_dict, metrics_type):
    dfs = []
    for alg in dir_dict:
        if alg =="pretrained" or 'Multitask' in alg:
            continue
        print(alg)
        df1 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=True)
        df2 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=False)
        df = pd.concat([df1, df2])
        df['task_id'] = df['task_id'].astype(int)
        df['at_task'] = df['at_task'].astype(int)
        df['step'] = df['step'].astype(int)
        df = df[df.task_id == df.at_task]
        if "task_detail" in  df.columns:
            # if df.groupby(["task_name", "task_detail"])["em"].transform(max) == 0.0:
            #     print("here")
            #     idx = df.groupby(["task_name", "task_detail"])["em"].transform(max)
            idx = df.groupby(["task_name", "task_detail"])[metrics_type].transform(max) == df[metrics_type]
        else:
            print("here")
            idx = df.groupby(["task_name"])["em"].transform(max) == df["em"]

        df = df[idx]                      
        df["alg"] = alg
        dfs.append(df)
    all_instant_df = pd.concat(dfs)
    all_instant_df = all_instant_df[[metrics_type, "task_name", "task_detail",  "alg", "task_id"]]
    all_instant_df = all_instant_df.rename(columns={metrics_type: f'instant-{metrics_type}', 
                                                      'alg': 'model', 
                                                      'task_name': 'dataset', 
                                                      'task_detail': 'task',
                                                    "task_id": "order"
                                                   })
    return all_instant_df


In [35]:
instant_auc_df = get_instant_df(dir_dict, metrics_type='auc')
instant_f1_df = get_instant_df(dir_dict, metrics_type='f1')

BART-BiHNet+Reg
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-BiHNet+EWC
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-Adapter-Vanilla
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-BiHNet+Vanilla
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-BiHNet

In [37]:
instant_df = instant_auc_df.merge(instant_f1_df, on=['model', 'dataset', 'task', 'order'])

In [38]:
instant_df.groupby("model")['instant-f1'].mean()

model
BART-Adapter-Vanilla    0.419515
BART-BiHNet+EWC         0.421098
BART-BiHNet+Reg         0.404110
BART-BiHNet+Vanilla     0.418059
Name: instant-f1, dtype: float64

In [39]:
instant_df.groupby("model")['instant-auc'].mean()

model
BART-Adapter-Vanilla    0.885684
BART-BiHNet+EWC         0.886328
BART-BiHNet+Reg         0.884639
BART-BiHNet+Vanilla     0.883828
Name: instant-auc, dtype: float64

# combine two tables

In [40]:
final_instant_df = final_df.merge(instant_df, on=['model', 'dataset', 'task'], how='left')


In [46]:
final_instant_df['model_order'] = final_instant_df['model'].replace(model_mapping)
final_instant_df = final_instant_df.sort_values(['order', 'model_order'])


In [47]:
final_instant_df = final_instant_df[['order', 'dataset', 'task', 'model', 'final-f1', 'instant-f1', 'final-auc', 'instant-auc']]

# Latex

In [48]:
latex_table =final_instant_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rlllrrrr}
\toprule
 order &         dataset &                     task &                  model &  final-f1 &  instant-f1 &  final-auc &  instant-auc \\
\midrule
   0.0 &          jigsaw &                  obscene &   BART-Adapter-Vanilla &  0.040219 &    0.199005 &   0.664802 &     0.977025 \\
   0.0 &          jigsaw &                  obscene &    BART-BiHNet+Vanilla &  0.026471 &    0.194175 &   0.726092 &     0.979034 \\
   0.0 &          jigsaw &                  obscene &        BART-BiHNet+Reg &  0.117117 &    0.208877 &   0.946478 &     0.978208 \\
   0.0 &          jigsaw &                  obscene &        BART-BiHNet+EWC &  0.035088 &    0.298387 &   0.649704 &     0.976722 \\
   1.0 &             ucc &    generalisation\_unfair &   BART-Adapter-Vanilla &  0.141791 &    0.256198 &   0.628530 &     0.860923 \\
   1.0 &             ucc &    generalisation\_unfair &    BART-BiHNet+Vanilla &  0.107817 &    0.222222 &   0.706271 &     0.853472 \\
   1.0 &        